In [10]:
import { RunnableSequence } from '@langchain/core/runnables';
import { PromptTemplate } from '@langchain/core/prompts';
import { OpenAI } from "@langchain/openai";
import { StringOutputParser ,StructuredOutputParser} from "@langchain/core/output_parsers";
import { z } from 'zod'

// const outputParser = new StringOutputParser()

const outputParser = StructuredOutputParser.fromZodSchema(z.object({
  answer: z.string().describe("answer to the user's question"),
  sources: z.array(z.string()).describe(
    "sources used to answer the question, should be websites.",
  ),
}));

const formatInstructions = outputParser.getFormatInstructions();

// const promptTemplate = PromptTemplate.fromTemplate(
//   "{question}",
// );

const promptTemplate = new PromptTemplate({
  template:
    "Answer the users question as best as possible.\n{format_instructions}\n{question}",
  inputVariables: ["question"],
  partialVariables: { format_instructions: formatInstructions },
});

const model = new OpenAI({
  configuration: {
      baseURL: "https://api.shellgpt.top/v1",
  }
});

const chain = RunnableSequence.from([promptTemplate, model, outputParser]);

await chain.invoke({
  question:'为什么地球是圆的'
})

{
  answer: "地球是圆的是因为它受到了自身的重力作用，使得它的形状更趋于球形。这是由于地球的自转和它的内部结构所造成的。",
  sources: [
    "https://zh.wikipedia.org/wiki/%E5%9C%B0%E7%90%83%E7%9A%84%E5%BD%A2%E7%8A%B6",
    "https://www.zhihu.com/question/20128893"
  ]
}